# Stack Overflow - Clustering

This notebook starts exploring the data of the Kaggle Stack Overflow question competition.
The competition is here: https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow

The goal was to build a classifier that redicts whether or not a question will be "closed" given the question as submitted.
Questions can be closed because they are: "off-topic", "unclear", "too broad" or "opinion based". There is also "duplicate" but that has been omitted from the data set. Currently 6% of the questions endup closed. 

In this first pass, we will see if clustering would be helpful. If time permits, we will run the competition next week. 

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import seaborn
from matplotlib import pyplot as plt
%matplotlib inline

In [4]:
data = pd.read_csv("train-sample.csv", index_col=0)
print (data.shape)

(140272, 14)


In [6]:
data.head(2)

,PostCreationDate,OwnerUserId,OwnerCreationDate,ReputationAtPostCreation,OwnerUndeletedAnswerCountAtPostTime,Title,BodyMarkdown,Tag1,Tag2,Tag3,Tag4,Tag5,PostClosedDate,OpenStatus
PostId,,,,,,,,,,,,,,
6046168,05/18/2011 14:14:05,543315,09/17/2010 10:15:06,1,2,For Mongodb is it better to reference an objec...,I am building a corpus of indexed sentences in...,mongodb,NaN,NaN,NaN,NaN,NaN,open
4873911,02/02/2011 11:30:10,465076,10/03/2010 09:30:58,192,24,How to insert schemalocation in a xml document...,i create a xml document with JAXP and search a...,dom,xsd,jaxp,NaN,NaN,NaN,open


The data has the following features
- PostId - ID number of the question
- PostCreationDate - Date and time the question was asked
- OwnerUserId - ID number of the question asker
- OwnerCreationDate - Date and time the question asker's account was created
- ReputationAtPostCreation - Reputation of the question asker at the time the question was asked
- OwnerUndeletedAnswerCountAtPostTime - Number of answers (to other questions) that the question asker had written at the time the question was asked
- Title - Title of the question
- BodyMarkdown - Body of the question
- Tag1 through Tag5 - Topical tags applied to the question
- PostClosedDate - Date and time the question was closed (if it was closed; not in test set)
- OpenStatus - 1 indicates open, 0 indicates closed (to be predicted in test set)

### Tags
One obvious candidate for clustering would be the vast amount of tags we have.

In [8]:
%%time
data['tags'] = data.apply(lambda x: " ".join(set(str(x['Tag%d' % i]) for i in range(1, 6))), axis=1)
data['tags'] = data['tags'].str.replace('nan', '').str.replace("  ", " ").str.strip()

Wall time: 7.92 s


In [10]:
tags = set([tag for tags in data.tags.values for tag in tags.split()])
print ("We have", len(tags), "tags.")

We have 18309 tags.


That is a lot. We could maybe cluster these, and then feed the cluster category into the model, rather than an ID for one of the 18K tags.  We could use `CountVectorizer` for this. Remember that 'CountVectorizer' converts a collection of text documents to a matrix of token counts.

In [11]:
n_clusters = 10
# N = 10000  # only use the first N documents for clustering (to speed up computations)
# data = data.iloc[:N]

In [12]:
%%time

# Learn the vocabulary dictionary and return term-document matrix.
cv = CountVectorizer(stop_words='english', ngram_range=(1, 1), max_features=5000, min_df=10, max_df=.95, binary=True)
X = cv.fit_transform(data.tags)

Wall time: 1.09 s


#### Clustering tags by their documents

In [14]:
X

<140272x3235 sparse matrix of type '<class 'numpy.int64'>'
	with 387519 stored elements in Compressed Sparse Row format>

In [15]:
XT = X.T
XT = StandardScaler().fit_transform(XT.toarray().astype(float))
XT

array([[-0.0175845 , -0.02487208, -0.03934447, ..., -0.02487208,
        -0.03046667, -0.03046667],
       [-0.0175845 , -0.02487208, -0.03934447, ..., -0.02487208,
        -0.03046667, -0.03046667],
       [-0.0175845 , -0.02487208, -0.03934447, ..., -0.02487208,
        -0.03046667, -0.03046667],
       ..., 
       [-0.0175845 , -0.02487208, -0.03934447, ..., -0.02487208,
        -0.03046667, -0.03046667],
       [-0.0175845 , -0.02487208, -0.03934447, ..., -0.02487208,
        -0.03046667, -0.03046667],
       [-0.0175845 , -0.02487208, -0.03934447, ..., -0.02487208,
        -0.03046667, -0.03046667]])

In [17]:
%%time
model = KMeans(n_clusters=n_clusters)
model.fit(XT)

Wall time: 4min 55s


In [19]:
np.unique(model.predict(XT), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([   1, 3224,    1,    1,    1,    1,    1,    1,    1,    3], dtype=int64))

Hmm, that didn't quite work: almost all tags got into one cluster, and the other clusters just got one tag. (Note we have less tags since we only used part of the data.)

#### Clustering documents by their tags

One other way we could do, is clustering the documents according to their tags, and then use their document cluster to feed into the model.

In [20]:
%%time
model.fit(X)

Wall time: 34.8 s


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [21]:
np.unique(model.predict(X), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([  300, 10180, 12392,  1821,  5676,  7338, 11996,  3650, 82943,  3976], dtype=int64))

Much better! Let's inspect the most common tags in these clusters.

In [26]:
top = 12
print ("Top %d tags per cluster:" % top)
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = cv.get_feature_names()
for i in range(n_clusters):
    print ("Cluster %d:" % i),
    for ind in order_centroids[i, :top]:
        print (terms[ind]),
    print ()

Top 12 tags per cluster:
Cluster 0:
coding
style
java
php
css
design
code
net
conventions
javascript
python
naming

Cluster 1:
net
asp
mvc
vb
sql
jquery
server
javascript
web
ajax
linq
framework

Cluster 2:
php
mysql
html
javascript
php5
jquery
arrays
ajax
codeigniter
wordpress
sql
database

Cluster 3:
visual
studio
2010
net
2008
asp
windows
sql
vb
2005
server
debugging

Cluster 4:
iphone
objective
ios
xcode
ipad
sdk
cocoa
touch
ios5
uitableview
core
android

Cluster 5:
javascript
jquery
html
ajax
html5
google
css
php
js
events
json
internet

Cluster 6:
java
android
ee
eclipse
swing
web
spring
xml
hibernate
jsp
php
javascript

Cluster 7:
ruby
rails
activerecord
python
jquery
heroku
mysql
php
rubygems
rspec
ajax
javascript

Cluster 8:
android
python
sql
jquery
windows
linux
server
mysql
google
database
facebook
web

Cluster 9:
html
css
jquery
javascript
internet
explorer
html5
css3
php
div
browser
image



Seems like there are at least a front-end cluster, a database cluster, and a mobile cluster.  Note that, since we clustered documents, not tags, a tag might be associated with several clusters.

In [30]:
data['tag_cluster'] = model.predict(X)
data.tag_cluster


PostId
6046168     8
4873911     8
3311559     8
9990413     2
10421966    8
8616154     8
1520973     5
5528942     8
4344698     8
7910832     8
11610237    8
9131744     8
2047987     8
8341885     5
11200627    1
6984871     8
8589517     8
8784855     3
8548243     8
5182954     8
4306229     8
6508144     8
11157471    8
7852524     1
11330139    8
10912452    8
2037737     8
7489468     8
7039786     8
4563508     5
           ..
8252258     8
5660600     1
6757001     1
10471141    8
8965603     6
2021996     2
11069552    8
3091804     8
4518655     6
9312476     6
11722379    8
1528        8
6632262     9
7411454     8
6132175     8
8084479     8
4986674     8
4498482     5
7821639     8
7678462     8
11006475    8
7437964     1
5320699     2
9127782     8
11487517    6
2982729     8
8809105     7
10674791    1
3997045     9
11570849    2
Name: tag_cluster, dtype: int32

In [32]:
#s = data.groupby('tag_cluster', 'open').OpenStatus.mean().sort(inplace=False)
#f = s.plot(kind='barh')

The difference in closed posts per cluster suggests we could try extracting some value from this.  We leave it as an exercise to verify if adding these clusters to your feature matrix indeed leads to a higher preduction accuracy.

#### Jaccard distance

The [_Jaccard index_](https://en.wikipedia.org/wiki/Jaccard_index) is a similarity metric between text documents. It measures how many words two documents have in common, as a fraction of the total number of distinct words in both documents.

$$\text{Jaccard index} = \frac{ |A \cap B | }{ |A \cup B| }$$

We could make a Jaccard matrix $J$, with pairwise similarities $J_{ij}$ as entries.
- `J[i, j]` = Jaccard similarity between doc _i_ and _j_ (between 0 and 1)
- `J[i, i]` = 1, obviously, and
- `J[i, j]` = `J[i, j]`, i.e., the matrix is symmetric.

We could also define the _Jaccard distance_, which has $D_{ii} = 0$ for identical documents, and bigger values as the documents have less words in common.  We define: $D = 1 - J,$ which has values between 0 and 1.

We could also use this for comparing our tags: how many documents do two tages have in common?

In [14]:
%%time
cv = CountVectorizer(stop_words='english', ngram_range=(1, 1), max_features=5000,
                     min_df=10, max_df=.95, binary=True)  # binary=True is important!
X = cv.fit_transform(data.tags)

CPU times: user 911 ms, sys: 18.8 ms, total: 930 ms
Wall time: 922 ms


Compute the Jaccard matrix and the distance matrix.

Note that we could compute

    U = [[n_docs[i] + n_docs[j] - I[i, j] for i in xrange(n_tags)] 
         for j in xrange(n_tags)]  # this is slow

but that is very slow. We use `numpy`'s vectorization and broadcasting instead.

In [15]:
n_tags = X.shape[1]  # number of tags
I = X.T.dot(X).toarray()  # X-transposed times X gives a tag x tag matrix with the # of docs in common
n_docs = np.diag(I)  # number of docs per tag
N = n_docs.reshape(n_tags, 1) * np.ones(n_tags)  # number of docs broadcasted over the entire row
U = N + N.T - I  # total distinct docs = n_docs_i + n_docs_j - words in common
J = I / U.astype(float)
D = 1 - J

Let's pick a few random tags and see what the closest tags are.

In [16]:
top = 10
tags = np.array(cv.get_feature_names())
for no in np.random.choice(n_tags, top, replace=False):  # pick 10 random tags
    print "%-18s:" % tags[no], " ".join(tags[D[no].argsort()[:top]])

paste             : paste copy cut clipboard pdt filepath filenames neo4j contextmenu vi
subclass          : subclass nib uilabel nsoperation covariance nsview extends nstableview pointer generic
tail              : tail recursion finite factorial best ocaml state append distribution logging
thrift            : thrift buffers protocol asio rpc weblogic cassandra agile compatibility boost
fragment          : fragment identifier changes orientation mapview honeycomb adapter osgi escaping state
launch            : launch condition installshield manifest orientation components purchase exe timeout device
fiddler           : fiddler firebug localhost playframework https proxy cookies firefox session post
arm               : arm embedded vlc microcontroller cpu floating x86 pi avr nfs
ajax              : ajax jquery javascript json asp php mvc net html forms
entitymanager     : entitymanager jpa ejb nullpointerexception hibernate generics netbeans entity spring java


Makes sense. Some tags have more meaning than others, I expect. Note that this is not a partitioning, as we only have a distance between words.

We could also apply `KMeans` to this Jaccard matrix, as the entries are indeed distances.

In [17]:
model.fit(D)
print "Top %d tags per cluster:" % top
order_centroids = model.cluster_centers_.argsort()
for i in range(n_clusters):
    print "Cluster %d:" % i, " ".join(tags[order_centroids[i, :top]])

Top 10 tags per cluster:
Cluster 0: objective iphone ios cocoa touch xcode memory ipad ios5 leaks
Cluster 1: studio visual 2010 castle source windsor protocol buffers setter getter
Cluster 2: internet machine js explorer computer intelligence questions interview coding mod
Cluster 3: processing gd image manipulation parallel transparency bit png imagemagick alpha
Cluster 4: exc bad array multidimensional access casting overloading interface type operator
Cluster 5: cream sandwich ice honeycomb softkeyboard decompiling launcher spinner htc fullscreen
Cluster 6: command ruby line amazon programming rails bash scripting shell languages
Cluster 7: asp net binding dependency wpf mvc mvvm injection data xaml
Cluster 8: latitude longitude area gprs distance circle spatial polygon mapkit coordinates
Cluster 9: intellij idea server sql google tips tricks engine dynamics crm


That looks quite promising.

Unfortunately, the model has put a lot of tags into the same cluster again:

In [18]:
np.unique(model.predict(D), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32),
 array([  37,   92, 2669,   35,   84,    3,  176,   28,    2,  109]))

<hr>
## Exercise
- Add the cluster information to your feature set and train a model on your data. Does the inclusion of these clusters indeed imporve prediction accuracy?  (Don't forget to cross-validate.)
- Try different values for `n_clusters` and see if you could find a good value using the _elbow method_.
- We have looked at clustering tags by documents, clustering documents by tags, and at the Jaccard distance between two tags using the number of documents they have in common.  Could you think of other ways of clustering tags or documents?  Try to implement these and see how effective they are.